# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
source activate pydeck-ee
jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import requests
import ee

## Authentication

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create Map

Next it's time to create a map. Here we create an `ee.Image` object

In [ ]:
# Initialize objects
ee_layers = []
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)

In [4]:
# %%
# Add Earth Engine dataset
# Use an elevation dataset and terrain functions to create
# a custom visualization of topography.

# Load a global elevation image.
elev = ee.Image('USGS/GMTED2010')

# Zoom to an area of interest.
view_state = pdk.ViewState(longitude=-121.069, latitude=50.709, zoom=6)

# Add the elevation to the map.
ee_layers.append(EarthEngineLayer(ee_object=elev, vis_params={}))

# Use the terrain algorithms to compute a hillshade with 8-bit values.
shade = ee.Terrain.hillshade(elev)
ee_layers.append(EarthEngineLayer(ee_object=shade, vis_params={}))

# Create a "sea" variable to be used for cartographic purposes
sea = elev.lte(0)
ee_layers.append(EarthEngineLayer(ee_object=sea.mask(sea), vis_params={'palette':'000022'}))

# Create a custom elevation palette from hex strings.
elevationPalette = ['006600', '002200', 'fff700', 'ab7634', 'c4d0ff', 'ffffff']
# Use these visualization parameters, customized by location.
visParams = {'min': 1, 'max': 3000, 'palette': elevationPalette}

# Create a mosaic of the sea and the elevation data
visualized = ee.ImageCollection([
  # Mask the elevation to get only land
  elev.mask(sea.Not()).visualize(**visParams),
  # Use the sea mask directly to display sea.
  sea.mask(sea).visualize(**{'palette':'000022'})
]).mosaic()

# Note that the visualization image doesn't require visualization parameters.
ee_layers.append(EarthEngineLayer(ee_object=visualized, vis_params={}))

# Convert the visualized elevation to HSV, first converting to [0, 1] data.
hsv = visualized.divide(255).rgbToHsv()
# Select only the hue and saturation bands.
hs = hsv.select(0, 1)
# Convert the hillshade to [0, 1] data, as expected by the HSV algorithm.
v = shade.divide(255)
# Create a visualization image by converting back to RGB from HSV.
# Note the cast to byte in order to export the image correctly.
rgb = hs.addBands(v).hsvToRgb().multiply(255).byte()
ee_layers.append(EarthEngineLayer(ee_object=rgb, vis_params={}))





Then just pass these layers to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [8]:
r = pdk.Deck(layers=ee_layers, initial_view_state=view_state)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…